In [ ]:
%cd ..

In [ ]:
from virtuoso.dataset import ScorePerformDataset, FeatureCollate
from pathlib import Path
from virtuoso.pyScoreParser import data_for_training as dft, data_class
import numpy as np

In [ ]:
dataset_path = '/home/teo/userdata/datasets/chopin_cleaned/'

In [ ]:
dataset = data_class.YamahaDataset(dataset_path, save=False, features_only=True)


In [ ]:
aligned_onset_deviations = []
average_deviaiton_list = []
ratio_list = []
for piece in dataset.pieces:
  for perform in piece.performances:
    len_notes =  len(perform.perform_features['align_matched'])
    num_aligned_notes = sum(perform.perform_features['align_matched'])
    if len_notes - num_aligned_notes > 800:
      continue
    if len_notes > num_aligned_notes * 1.5:
      continue
    align_matched = np.asarray(perform.perform_features['align_matched']) == 1
    valid_onset_deviation = np.abs(perform.perform_features['onset_deviation'])[align_matched]
    max_deviation = max(valid_onset_deviation)
    average_deviation = valid_onset_deviation.mean()
    over_dev_ratio = np.sum(valid_onset_deviation>1) / np.sum(align_matched)
    too_large_dev_num = np.sum(valid_onset_deviation>4)
#     if over_dev_ratio > 0.005:
#       print(perform.midi_path)
#       continue
    if max_deviation > 4:
      print(max_deviation, average_deviation, over_dev_ratio)
      print(too_large_dev_num)
    aligned_onset_deviations += valid_onset_deviation.tolist()

    average_deviaiton_list.append(average_deviation)
    ratio_list.append(over_dev_ratio)

In [ ]:
import plotly.express as px

px.histogram(average_deviaiton_list)

In [ ]:
px.histogram(ratio_list)

In [ ]:
px.histogram(aligned_onset_deviations)

In [ ]:
train_set = ScorePerformDataset(args.data_path, 
                                type="train", 
                                len_slice=args.len_slice, 
                                len_graph_slice=args.len_graph_slice, 
                                graph_keys=args.graph_keys, 
                                hier_type=curr_type)
